Organization:
- Work
  - 1 test: defining functions for part 1, testing on test input
  - 1 run: getting answer for part 1
  - 2 test: ...
  - 2 run: ...
- Utilities: functions I think might help parse general inputs
- Inputs: where I define the test (_t_) and problem (_s_) inputs

# Work

## 1 test

Idea: recursive calls to get the values of monkeys. Implemented with a dictionary (for monkey memoization).

In [40]:
# Parse the input, making sure it looks like we expect
def parse(line):
    out = line.split(': ')
    try:
        out[1] = int(out[1])
    except:
        assert ('+' in out[1]) or ('-' in out[1]) or ('*' in out[1]) or ('/' in out[1])
    return tuple(out)

monke = [parse(line) for line in split(t)]
monke

[('root', 'pppw + sjmn'),
 ('dbpl', 5),
 ('cczh', 'sllz + lgvd'),
 ('zczc', 2),
 ('ptdq', 'humn - dvpt'),
 ('dvpt', 3),
 ('lfqf', 4),
 ('humn', 5),
 ('ljgn', 2),
 ('sjmn', 'drzm * dbpl'),
 ('sllz', 4),
 ('pppw', 'cczh / lfqf'),
 ('lgvd', 'ljgn * ptdq'),
 ('drzm', 'hmdt - zczc'),
 ('hmdt', 32)]

In [41]:
# Make a dictionary of the monkey values (or expressions)
monkey_values = dict(monke)

In [42]:
# A recursive function to get the value of a given monkey
def value(monkey):
    global monkey_values
    
    expr = monkey_values[monkey]
    
    # Base case: value exists
    if isinstance(expr, int):
        return expr
    
    # Otherwise, recurse
    monkeyL, op, monkeyR = tuple(expr.split())
    L, R = value(monkeyL), value(monkeyR)
    monkey_values[monkey] = int(eval(f'L {op} R'))
    
    return monkey_values[monkey]

In [43]:
value('root')

152

## 1 run

In [45]:
monke = [parse(line) for line in split(s)]

monkey_values = dict(monke)

value('root')

78342931359552

## 2 test

Run the same thing as the previous part on each of the monkeys that **root** calls, except the monkey **humn** should return **x** so that all the monkey's above it have expressions involving **x**. Then solve it somewhat manually, since setting up a sort of reverse recursion for this might be a pain. This should be faster.

In [113]:
monke = [parse(line) for line in split(t)]

monkey_values = dict(monke)
print(f'Root expression: {monkey_values["root"]}')
monkey_values['humn'] = 'x'
del monkey_values['root']

Root expression: pppw + sjmn


In [114]:
# A recursive function to get the value of a given monkey, allowing for humn = x and xs throughout
def value_humn(monkey):
    global monkey_values
    
    expr = monkey_values[monkey]
    
    # Base case: value exists or it's the human
    if isinstance(expr, int) or monkey == 'humn':
        return expr
    
    # Otherwise, recurse
    monkeyL, op, monkeyR = tuple(expr.split())
    L, R = value_humn(monkeyL), value_humn(monkeyR)
    
    # For the math, form the expression and try evaluating it
    # Evaluation might fail because of an x
    expr = f'({L} {op} {R})'
    try:
        expr = int(eval(expr))
    except:
        pass
    monkey_values[monkey] = expr
    
    return expr

In [115]:
expr = value_humn('pppw')
expr

'((4 + (2 * (x - 3))) / 4)'

In [116]:
val = value_humn('sjmn')
val

150

WolframAlpha solves it to get **x=301**

Or in Python, we know this is linear so write **y = mx + b** and use **x=0,1** to solve for **m,b**

In [118]:
# The function to solve equal to zero
f = lambda x: eval(expr.replace('x', str(x))) - val

In [123]:
# Solve
b = f(0)
m = f(1) - b
-b/m

301.0

In [124]:
# Check
f(301)

0.0

## 2 run

In [130]:
monke = [parse(line) for line in split(s)]

monkey_values = dict(monke)
print(f'Root expression: {monkey_values["root"]}')
monkey_values['humn'] = 'x'
del monkey_values['root']

Root expression: lvvf + rqgq


In [131]:
expr = value_humn('lvvf')
expr

'(8 * (6862813426220 - ((((141 + ((((((((((((((((((((165 + ((346 + (((29 + (((2 * (((((((573 + ((((((2 * (982 + (2 * ((((2 * (45 + ((((((528 + (((614 + ((((577 + ((x - 961) * 29)) / 7) - 507) * 9)) + 281) / 2)) / 2) - 153) * 3) - 769) / 5))) - 826) / 2) + 147)))) - 158) / 2) - 104) + 835) / 3)) * 8) - 890) / 2) + 124) * 2) - 233)) + 752) / 2)) * 2) - 675)) / 3)) / 2) - 866) * 3) - 999) * 2) + 404) / 2) - 56) * 2) + 944) / 2) - 822) * 2) + 964) / 4) - 459) * 2) + 639) + 546)) / 5) + 378) / 5)))'

In [132]:
val = value_humn('rqgq')
val

23440423968672

In [133]:
# The function to solve equal to zero
f = lambda x: eval(expr.replace('x', str(x))) - val

In [134]:
# Solve: lmao precision issues!
b = f(0)
m = f(1) - b
-b/m

3295537382559.77

Scipy will do it for me

In [156]:
from scipy.optimize import bisect

In [164]:
bisect(f, 1000000000000, 10000000000000)

3296135418820.0

In [165]:
f(3296135418820)

0.0

# Utilities

In [1]:
# Remove initial/final \n characters
def clean(s):
    return s[1:-1]

# Split at \n characters
# If there are \n\n characters, split into blocks too
def split(s, block_char = '\n\n', line_char = '\n'):
    out = [block.split(line_char) for block in clean(s).split(block_char)]
    if len(out) == 1:
        return out[0]
    else:
        return out

# Apply a function(s) to a list or "block" data (2-level list)
def apply_func(data, func, nested=False):
    if not isinstance(func, list):
        func = [func]
        
    def _func(x):
        for f in func:
            x = f(x)
        return x
        
    if nested:
        return [[_func(x) for x in block] for block in data]
    else:
        return [_func(x) for x in data]

# Split, parsing everything as ints
def split_int(s):
    return apply_func(split(s), int)

# Split, parsing everything as float
def split_float(s):
    return apply_func(split(s), float)

# Inputs

In [2]:
t = """
root: pppw + sjmn
...
hmdt: 32
"""

In [3]:
s = """
lfrh: 2
...
zfqn: zdcs * hvrb
"""